In [24]:
from azure.storage.blob import BlobServiceClient
import os


# Azure storage connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=sapocopenai;AccountKey=H30mVmhmyKpFmLUkFSKpDO3CUe+jbcG8aGZ8TgCRNTQj5Ac5HB+649BzYypyo9eW0W9BRy9Z0oFr+ASt6hAVYw==;EndpointSuffix=core.windows.net"
container_name = "bulk-upload"
local_folder = "files"

# Create a BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get the container client
container_client = blob_service_client.get_container_client(container_name)

# Upload files
for root, dirs, files in os.walk(local_folder):
    for file in files:
        blob_client = container_client.get_blob_client(os.path.join(root, file))
        with open(os.path.join(root, file), "rb") as data:
            blob_client.upload_blob(data)
        print(f"Uploaded {file} to container {container_name}")

Uploaded DSI_User_mannual_Kumo-AI (1).pdf to container bulk-upload
Uploaded MoM_26-10-23.txt to container bulk-upload
Uploaded MoM_30-10-23.txt to container bulk-upload


In [26]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SimpleField, SearchIndex, SearchableField, SearchField, SearchFieldDataType

# Azure Search credentials
search_service_name = "poc-openai-cogsrch"

admin_key = AzureKeyCredential("TBpoXA6at3L38HsBWSRHnZWfstC07Txt7kDulY7ccDAzSeAHslm9")
index_name = "akurapa_bulk"
# indexer_client = SearchIndexerClient(endpoint=f"https://{search_service_name}.search.windows.net", credential=admin_key)

# admin_key = "your_admin_key"
# index_name = "your_index_name"

# Initialize SearchIndexClient
search_service_endpoint = f"https://{search_service_name}.search.windows.net"
search_client = SearchIndexClient(endpoint=search_service_endpoint, credential=admin_key)

# Define the search index schema
fields = [
    SimpleField(name="id", type="Edm.String", key=True),
    SearchableField(name="content", type="Edm.String", searchable=True, filterable=False, sortable=False),
    SearchField(name="metadata_storage_path", type=SearchFieldDataType.String)
]

index = SearchIndex(name=index_name, fields=fields)

# Create the search index
search_client.create_index(index)
print(f"Index '{index_name}' created.")


Index 'akurapa_bulk' created.


In [27]:
from langchain.document_loaders import UnstructuredPDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

path = "2_sample_biology.pdf"
loader = PyPDFLoader(path)

doc = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap  = 200)
docs = text_splitter.split_documents(doc)

In [28]:
# docs

[Document(page_content='Case Report\nComplementary Effects of Surgery and Pexidartinib in the\nManagement of Patients with Complex Diffuse-TenosynovialGiant Cell Tumor\nNicholas M. Bernthal ,1R. Lor Randall ,2Lauren N. Zeitlinger ,2Erik J. Geiger ,1\nand John H. Healey3\n1Department of Orthopaedic Surgery, David Ge ﬀen School of Medicine at UCLA, 1225 15th Street, Suite 2100, Santa Monica, CA,\nUSA 90404\n2Department of Orthopaedic Surgery, University of California, Davis, 4860 Y Street, Suite 3800, Sacramento, CA, USA 95817\n3Department of Surgery, Orthopaedic Service, Memorial Sloan Kettering Cancer Center, 1275 York Avenue, New York, NY, USA\n10065\nCorrespondence should be addressed to Nicholas M. Bernthal; nbernthal@mednet.ucla.edu\nReceived 20 June 2022; Revised 6 October 2022; Accepted 4 November 2022; Published 3 December 2022 \nAcademic Editor: Taketoshi Yasuda\nCopyright © 2022 Nicholas M. Bernthal et al. This is an open access article distributed under the Creative Commons A

In [29]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

admin_key = AzureKeyCredential("TBpoXA6at3L38HsBWSRHnZWfstC07Txt7kDulY7ccDAzSeAHslm9")
# index_name = "bulk-upload-testing"
index_name = "akurapa_bulk"
search_service_name = "poc-openai-cogsrch"
search_service_endpoint = f"https://{search_service_name}.search.windows.net"

# Initialize SearchClient
index_client = SearchClient(endpoint=search_service_endpoint, index_name=index_name, credential=admin_key)



In [30]:
documents = []
for i, k in enumerate(docs):
    document = {
        "id": str(i+1),
        "content": k.page_content,
        "metadata_storage_path": k.metadata['source']
    }
    documents.append(document)

In [31]:
index_client.upload_documents(documents=documents)
print("Documents uploaded to the index.")

Documents uploaded to the index.


In [32]:
results = index_client.search("Tenosynovial giant cell tumor (TGCT",search_fields=["content"], top=5)

In [33]:
for result in results:
    print(result)

{'id': '8', 'metadata_storage_path': '2_sample_biology.pdf', 'content': '2. Case Presentations\n2.1. Case 1-Hip. A 29-year-old male with no past medical\nhistory developed new onset left-lateral hip pain while run-ning in August 2019. After an unsuccessful trial of physical\n(a)\n (b)\n(c)\nFigure 1: Representative MRI slices from of a 29-year-old male with D-TGCT in the left hip at diagnosis. Coronal (a, b) and axial (c) MR\nimaging demonstrating severe arthritic changes of the left hip joint with erosions present (∗), (b) massive TGCT tumor burden running\nalong the anterior hip and into the pelvis adjacent to the iliacus muscle (arrowhead), and (c) tumor burden both posterior to the hip inthe ischiofemoral fossa and anterior to the psoas insertion (arrow).\nFigure 2: Histology images in the hip prior to pexidartinib\ntreatment. The pretreatment biopsy demonstrating a cellularproliferation of tumor cells composed of abundant hemosiderinladen macrophages, multinucleated giant cells, s